# HiPPO Matrices
---

## Table of Contents
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
    * [Translated Legendre (LegT)](#translated-legendre-legt)
        * [LegT](#legt)
        * [LMU](#lmu)
    * [Translated Laguerre (LagT)](#translated-laguerre-lagt)
    * [Scaled Legendre (LegS)](#scaled-legendre-legs)
    * [Fourier Basis](#fourier-basis)
        * [Fourier Recurrent Unit (FRU)](#fourier-recurrent-unit-fru)
        * [Truncated Fourier (FouT)](#truncated-fourier-fout)
        * [Fourier With Decay (FourD)](#fourier-with-decay-fourd)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
    * [Testing Forward Euler on GBT matrices](#testing-forward-euler-transform-for-lti-and-lsi)
    * [Testing Backward Euler on GBT matrices](#testing-backward-euler-transform-for-lti-and-lsi-on-legs-matrices)
    * [Testing Bidirectional on GBT matrices](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on GBT matrices](#testing-zoh-transform-for-lti-and-lsi-on-legs-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
    * [Testing Forward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-forward-euler-transform)
    * [Testing Backward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-backward-euler-transform)
    * [Testing Bidirectional on HiPPO Operators](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on HiPPO Operators](#testing-lti-and-lsi-operators-with-zoh-transform)
---


## Load Packages

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../../../"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
## import packages
import math

import jax
import jax.numpy as jnp
import requests
from flax import linen as jnn
from jax.nn.initializers import lecun_normal, uniform
from jax.numpy.linalg import eig, inv, matrix_power
from jax.scipy.signal import convolve
from scipy import linalg as la
from scipy import signal
from scipy import special as ss

from src.data.process import moving_window, rolling_window

# import modules
from src.models.hippo.gu_transition import GuTransMatrix
from src.models.hippo.unroll import (
    basis,
    measure,
    variable_unroll_matrix,
    variable_unroll_matrix_sequential,
)

print(jax.devices())
print(f"The Device: {jax.lib.xla_bridge.get_backend().platform}")

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
The Device: gpu


In [3]:
from functools import partial
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from einops import rearrange, reduce, repeat

print(f"MPS enabled: {torch.backends.mps.is_available()}")

MPS enabled: False


In [4]:
torch.set_printoptions(linewidth=150)
np.set_printoptions(linewidth=150)
jnp.set_printoptions(linewidth=150)

In [5]:
seed = 1701
key = jax.random.PRNGKey(seed)

In [6]:
num_copies = 5
rng, key2, key3, key4, key5 = jax.random.split(key, num=num_copies)

## Instantiate The HiPPO Matrix

In [7]:
class TransMatrix:
    def __init__(
        self,
        N: int,
        measure: str = "legs",
        lambda_n: float = 1.0,
        alpha: float = 0.0,
        beta: float = 1.0,
        dtype: Any = jnp.float32,
    ):
        """
        Instantiates the HiPPO matrix of a given order using a particular measure.
        Args:
            N (int): Order of coefficients to describe the orthogonal polynomial that is the HiPPO projection.
            v (str): choose between this repo's implementation or hazy research's implementation.
            measure (str):
                choose between
                    - HiPPO w/ Translated Legendre (LegT) - legt
                    - HiPPO w/ Translated Laguerre (LagT) - lagt
                    - HiPPO w/ Scaled Legendre (LegS) - legs
                    - HiPPO w/ Fourier basis
                        - FRU: Fourier Recurrent Unit - fru
                        - FouT: Translated Fourier - fout
                        - FourD: Fourier Decay - fourd
            lambda_n (int): The amount of tilt applied to the HiPPO-LegS basis, determines between LegS and LMU.
            alpha (float): The order of the Laguerre basis.
            beta (float): The scale of the Laguerre basis.

        Returns:
            A (jnp.ndarray): The HiPPO matrix multiplied by -1.
            B (jnp.ndarray): The other corresponding state space matrix.

        """
        A = None
        B = None
        if measure in ["legt", "lmu"]:
            if measure == "legt":
                assert lambda_n == 1.0
            elif measure == "lmu":
                assert lambda_n == 2.0
            else:
                raise ValueError("Invalid lambda_n for HiPPO type 'legt' or 'lmu")

            A, B = self.build_LegT(N=N, lambda_n=lambda_n, dtype=dtype)

        elif measure == "lagt":
            A, B = self.build_LagT(alpha=alpha, beta=beta, N=N, dtype=dtype)

        elif measure == "legs":
            A, B = self.build_LegS(N=N, dtype=dtype)

        elif measure in ["fout", "fru", "foud"]:
            A, B = self.build_Fourier(N=N, fourier_type=measure, dtype=dtype)

        elif measure == "random":
            A = jnp.random.randn(N, N) / N
            B = jnp.random.randn(N, 1)

        elif measure == "diagonal":
            A = -jnp.diag(jnp.exp(jnp.random.randn(N)))
            B = jnp.random.randn(N, 1)

        else:
            raise ValueError("Invalid HiPPO type")

        self.A = (A.copy()).astype(dtype)
        self.B = (B.copy()).astype(dtype)

    # Translated Legendre (LegT) - vectorized
    @staticmethod
    def build_LegT(N, lambda_n=1, dtype=jnp.float32):
        """
        The, vectorized implementation of the, measure derived from the translated Legendre basis.

        Args:
            N (int): Order of coefficients to describe the orthogonal polynomial that is the HiPPO projection.
            legt_type (str): Choice between the two different tilts of basis.
                - legt: translated Legendre - 'legt'
                - lmu: Legendre Memory Unit - 'lmu'

        Returns:
            A (jnp.ndarray): The A HiPPO matrix.
            B (jnp.ndarray): The B HiPPO matrix.

        """
        q = jnp.arange(N, dtype=dtype)
        k, n = jnp.meshgrid(q, q)
        case = jnp.power(-1.0, (n - k))
        A = None
        B = None

        if lambda_n == 1:
            A_base = jnp.sqrt(2 * n + 1) * jnp.sqrt(2 * k + 1)
            pre_D = jnp.sqrt(jnp.diag(2 * q + 1))
            B = D = jnp.diag(pre_D)[:, None]
            A = jnp.where(
                k <= n, A_base, A_base * case
            )  # if n >= k, then case_2 * A_base is used, otherwise A_base

        elif lambda_n == 2:  # (jnp.sqrt(2*n+1) * jnp.power(-1, n)):
            A_base = 2 * n + 1
            B = jnp.diag((2 * q + 1) * jnp.power(-1, n))[:, None]
            A = jnp.where(
                k <= n, A_base * case, A_base
            )  # if n >= k, then case_2 * A_base is used, otherwise A_base

        return -A.astype(dtype), B.astype(dtype)

    # Translated Laguerre (LagT) - non-vectorized
    @staticmethod
    def build_LagT(alpha, beta, N, dtype=jnp.float32):
        """
        The, vectorized implementation of the, measure derived from the translated Laguerre basis.

        Args:
            alpha (float): The order of the Laguerre basis.
            beta (float): The scale of the Laguerre basis.
            N (int): Order of coefficients to describe the orthogonal polynomial that is the HiPPO projection.

        Returns:
            A (jnp.ndarray): The A HiPPO matrix.
            B (jnp.ndarray): The B HiPPO matrix.

        """
        L = jnp.exp(
            0.5
            * (ss.gammaln(jnp.arange(N) + alpha + 1) - ss.gammaln(jnp.arange(N) + 1))
        )
        inv_L = 1.0 / L[:, None]
        pre_A = (jnp.eye(N) * ((1 + beta) / 2)) + jnp.tril(jnp.ones((N, N)), -1)
        pre_B = ss.binom(alpha + jnp.arange(N), jnp.arange(N))[:, None]

        A = -inv_L * pre_A * L[None, :]
        B = (
            jnp.exp(-0.5 * ss.gammaln(1 - alpha))
            * jnp.power(beta, (1 - alpha) / 2)
            * inv_L
            * pre_B
        )

        return A.astype(dtype), B.astype(dtype)

    # Scaled Legendre (LegS) vectorized
    @staticmethod
    def build_LegS(N, dtype=jnp.float32):
        """
        The, vectorized implementation of the, measure derived from the Scaled Legendre basis.

        Args:
            N (int): Order of coefficients to describe the orthogonal polynomial that is the HiPPO projection.

        Returns:
            A (jnp.ndarray): The A HiPPO matrix.
            B (jnp.ndarray): The B HiPPO matrix.

        """
        q = jnp.arange(N, dtype=dtype)
        k, n = jnp.meshgrid(q, q)
        pre_D = jnp.sqrt(jnp.diag(2 * q + 1))
        B = D = jnp.diag(pre_D)[:, None]

        A_base = jnp.sqrt(2 * n + 1) * jnp.sqrt(2 * k + 1)

        A = jnp.where(n > k, A_base, jnp.where(n == k, n + 1, 0.0))

        return -A.astype(dtype), B.astype(dtype)

    # Fourier Basis OPs and functions - vectorized
    @staticmethod
    def build_Fourier(N, fourier_type="fru", dtype=jnp.float32):
        """
        Vectorized measure implementations derived from fourier basis.

        Args:
            N (int): Order of coefficients to describe the orthogonal polynomial that is the HiPPO projection.
            fourier_type (str): The type of Fourier measure.
                - FRU: Fourier Recurrent Unit - fru
                - FouT: truncated Fourier - fout
                - fouD: decayed Fourier - foud

        Returns:
            A (jnp.ndarray): The A HiPPO matrix.
            B (jnp.ndarray): The B HiPPO matrix.

        """
        A = jnp.diag(
            jnp.stack([jnp.zeros(N // 2), jnp.zeros(N // 2)], axis=-1).reshape(-1)[1:],
            1,
        )
        B = jnp.zeros(A.shape[1], dtype=dtype)

        B = B.at[0::2].set(jnp.sqrt(2))
        B = B.at[0].set(1)

        q = jnp.arange(A.shape[1], dtype=dtype)
        k, n = jnp.meshgrid(q, q)

        n_odd = n % 2 == 0
        k_odd = k % 2 == 0

        case_1 = (n == k) & (n == 0)
        case_2_3 = ((k == 0) & (n_odd)) | ((n == 0) & (k_odd))
        case_4 = (n_odd) & (k_odd)
        case_5 = (n - k == 1) & (k_odd)
        case_6 = (k - n == 1) & (n_odd)

        if fourier_type == "fru":  # Fourier Recurrent Unit (FRU) - vectorized
            A = jnp.where(
                case_1,
                -1.0,
                jnp.where(
                    case_2_3,
                    -jnp.sqrt(2),
                    jnp.where(
                        case_4,
                        -2,
                        jnp.where(
                            case_5,
                            jnp.pi * (n // 2),
                            jnp.where(case_6, -jnp.pi * (k // 2), 0.0),
                        ),
                    ),
                ),
            )

        elif fourier_type == "fout":  # truncated Fourier (FouT) - vectorized
            A = jnp.where(
                case_1,
                -1.0,
                jnp.where(
                    case_2_3,
                    -jnp.sqrt(2),
                    jnp.where(
                        case_4,
                        -2,
                        jnp.where(
                            case_5,
                            jnp.pi * (n // 2),
                            jnp.where(case_6, -jnp.pi * (k // 2), 0.0),
                        ),
                    ),
                ),
            )

            A = 2 * A
            B = 2 * B

        elif fourier_type == "foud":
            A = jnp.where(
                case_1,
                -1.0,
                jnp.where(
                    case_2_3,
                    -jnp.sqrt(2),
                    jnp.where(
                        case_4,
                        -2,
                        jnp.where(
                            case_5,
                            2 * jnp.pi * (n // 2),
                            jnp.where(case_6, 2 * -jnp.pi * (k // 2), 0.0),
                        ),
                    ),
                ),
            )

            A = 0.5 * A
            B = 0.5 * B

        B = B[:, None]

        return A.astype(dtype), B.astype(dtype)

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

In [8]:
num_of_coef = 8

## Translated Legendre (LegT)

### LegT

In [9]:
def test_LegT(N):
    legt_matrices = TransMatrix(N=N, measure="legt", lambda_n=1.0)
    A, B = legt_matrices.A, legt_matrices.B
    gu_legt_matrices = GuTransMatrix(N=N, measure="legt", lambda_n=1.0)
    gu_A, gu_B = gu_legt_matrices.A, gu_legt_matrices.B
    print(f"A:\n", A)
    print(f"Gu's A:\n", gu_A)
    print(f"B:\n", B)
    print(f"Gu's B:\n", gu_B)
    assert jnp.allclose(A, gu_A)
    assert jnp.allclose(B, gu_B)

In [10]:
test_LegT(N=num_of_coef)

A:
 [[ -1.          1.7320508  -2.2360678 ...   9.746794   -9.848858    9.949874 ]
 [ -1.7320508  -3.          3.872983  ... -16.88194    17.058722  -17.233686 ]
 [ -2.2360678  -3.872983   -4.999999  ...  21.79449   -22.022715   22.248592 ]
 ...
 [ -9.746794  -16.88194   -21.79449   ... -94.999985   95.99479   -96.97937  ]
 [ -9.848858  -17.058722  -22.022715  ... -95.99479   -97.         97.994896 ]
 [ -9.949874  -17.233686  -22.248592  ... -96.97937   -97.994896  -98.99999  ]]
Gu's A:
 [[ -1.          1.7320508  -2.2360678 ...   9.746794   -9.848858    9.949874 ]
 [ -1.7320508  -3.          3.872983  ... -16.88194    17.058722  -17.233686 ]
 [ -2.2360678  -3.872983   -4.999999  ...  21.79449   -22.022715   22.248592 ]
 ...
 [ -9.746794  -16.88194   -21.79449   ... -94.999985   95.99479   -96.97937  ]
 [ -9.848858  -17.058722  -22.022715  ... -95.99479   -97.         97.994896 ]
 [ -9.949874  -17.233686  -22.248592  ... -96.97937   -97.994896  -98.99999  ]]
B:
 [[1.       ]
 [1.732050

### LMU

In [11]:
def test_LMU(N):
    lmu_matrices = TransMatrix(
        N=N, measure="lmu", lambda_n=2.0
    )  # change lambda so resulting matrix is in the form of LMU
    A, B = lmu_matrices.A, lmu_matrices.B
    gu_lmu_matrices = GuTransMatrix(
        N=N, measure="lmu", lambda_n=2.0
    )  # change lambda so resulting matrix is in the form of LMU
    gu_A, gu_B = gu_lmu_matrices.A, gu_lmu_matrices.B
    print(f"A:\n", A)
    print(f"Gu's A:\n", gu_A)
    print(f"B:\n", B)
    print(f"Gu's B:\n", gu_B)
    assert jnp.allclose(A, gu_A)
    assert jnp.allclose(B, gu_B)

In [12]:
test_LMU(N=num_of_coef)

A:
 [[ -1.  -1.  -1. ...  -1.  -1.  -1.]
 [  3.  -3.  -3. ...  -3.  -3.  -3.]
 [ -5.   5.  -5. ...  -5.  -5.  -5.]
 ...
 [ 95. -95.  95. ... -95. -95. -95.]
 [-97.  97. -97. ...  97. -97. -97.]
 [ 99. -99.  99. ... -99.  99. -99.]]
Gu's A:
 [[ -1.  -1.  -1. ...  -1.  -1.  -1.]
 [  3.  -3.  -3. ...  -3.  -3.  -3.]
 [ -5.   5.  -5. ...  -5.  -5.  -5.]
 ...
 [ 95. -95.  95. ... -95. -95. -95.]
 [-97.  97. -97. ...  97. -97. -97.]
 [ 99. -99.  99. ... -99.  99. -99.]]
B:
 [[  1.]
 [ -3.]
 [  5.]
 [ -7.]
 [  9.]
 [-11.]
 [ 13.]
 [-15.]
 [ 17.]
 [-19.]
 [ 21.]
 [-23.]
 [ 25.]
 [-27.]
 [ 29.]
 [-31.]
 [ 33.]
 [-35.]
 [ 37.]
 [-39.]
 [ 41.]
 [-43.]
 [ 45.]
 [-47.]
 [ 49.]
 [-51.]
 [ 53.]
 [-55.]
 [ 57.]
 [-59.]
 [ 61.]
 [-63.]
 [ 65.]
 [-67.]
 [ 69.]
 [-71.]
 [ 73.]
 [-75.]
 [ 77.]
 [-79.]
 [ 81.]
 [-83.]
 [ 85.]
 [-87.]
 [ 89.]
 [-91.]
 [ 93.]
 [-95.]
 [ 97.]
 [-99.]]
Gu's B:
 [[  1.]
 [ -3.]
 [  5.]
 [ -7.]
 [  9.]
 [-11.]
 [ 13.]
 [-15.]
 [ 17.]
 [-19.]
 [ 21.]
 [-23.]
 [ 25.]
 [-27.]
 [ 29

## Translated Laguerre (LagT)

In [13]:
def test_LagT(N):
    lagt_matrices = TransMatrix(
        N=N,
        measure="lagt",
        alpha=0.0,  # change resulting tilt through alpha and beta
        beta=1.0,
    )  # change resulting tilt through alpha and beta
    A, B = lagt_matrices.A, lagt_matrices.B
    gu_lagt_matrices = GuTransMatrix(
        N=N,
        measure="lagt",
        alpha=0.0,  # change resulting tilt through alpha and beta
        beta=1.0,
    )  # change resulting tilt through alpha and beta
    gu_A, gu_B = gu_lagt_matrices.A, gu_lagt_matrices.B
    print(f"A:\n", A)
    print(f"Gu's A:\n", gu_A)
    print(f"B:\n", B)
    print(f"Gu's B:\n", gu_B)
    assert jnp.allclose(A, gu_A)
    assert jnp.allclose(B, gu_B)

In [14]:
test_LagT(N=num_of_coef)

A:
 [[-1.        -0.        -0.        ... -0.        -0.        -0.       ]
 [-1.        -1.        -0.        ... -0.        -0.        -0.       ]
 [-1.        -1.        -1.        ... -0.        -0.        -0.       ]
 ...
 [-1.0000018 -1.0000018 -1.0000018 ... -1.        -0.        -0.       ]
 [-1.000002  -1.000002  -1.000002  ... -1.0000002 -1.        -0.       ]
 [-0.9999969 -0.9999969 -0.9999969 ... -0.9999951 -0.9999949 -1.       ]]
Gu's A:
 [[-1.        -0.        -0.        ... -0.        -0.        -0.       ]
 [-1.        -1.        -0.        ... -0.        -0.        -0.       ]
 [-1.        -1.        -1.        ... -0.        -0.        -0.       ]
 ...
 [-1.0000018 -1.0000018 -1.0000018 ... -1.        -0.        -0.       ]
 [-1.000002  -1.000002  -1.000002  ... -1.0000002 -1.        -0.       ]
 [-0.9999969 -0.9999969 -0.9999969 ... -0.9999951 -0.9999949 -1.       ]]
B:
 [[1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.       

## Scaled Legendre (LegS)

In [15]:
def test_LegS(N):
    legs_matrices = TransMatrix(N=N, measure="legs")
    A, B = legs_matrices.A, legs_matrices.B
    gu_legs_matrices = GuTransMatrix(N=N, measure="legs")
    gu_A, gu_B = gu_legs_matrices.A, gu_legs_matrices.B
    print(f"A:\n", A)
    print(f"Gu's A:\n", gu_A)
    print(f"B:\n", B)
    print(f"Gu's B:\n", gu_B)
    assert jnp.allclose(A, gu_A)
    assert jnp.allclose(B, gu_B)

In [16]:
test_LegS(N=num_of_coef)

A:
 [[ -1.         -0.         -0.        ...  -0.         -0.         -0.       ]
 [ -1.7320508  -2.         -0.        ...  -0.         -0.         -0.       ]
 [ -2.2360678  -3.872983   -3.        ...  -0.         -0.         -0.       ]
 ...
 [ -9.746794  -16.88194   -21.79449   ... -48.         -0.         -0.       ]
 [ -9.848858  -17.058722  -22.022715  ... -95.99479   -49.         -0.       ]
 [ -9.949874  -17.233686  -22.248592  ... -96.97937   -97.994896  -50.       ]]
Gu's A:
 [[ -1.          0.          0.        ...   0.          0.          0.       ]
 [ -1.7320508  -1.9999999   0.        ...   0.          0.          0.       ]
 [ -2.2360678  -3.872983   -3.        ...   0.          0.          0.       ]
 ...
 [ -9.746794  -16.88194   -21.794497  ... -48.          0.          0.       ]
 [ -9.848858  -17.058722  -22.022718  ... -95.9948    -48.999996    0.       ]
 [ -9.949874  -17.233686  -22.248598  ... -96.97938   -97.99489   -50.       ]]
B:
 [[1.       ]
 [1.732050

## Fourier Basis

### Fourier Recurrent Unit (FRU)

In [17]:
def test_FRU(N):
    fru_matrices = TransMatrix(N=N, measure="fru")
    A, B = fru_matrices.A, fru_matrices.B
    gu_fru_matrices = GuTransMatrix(N=N, measure="fru")
    gu_A, gu_B = gu_fru_matrices.A, gu_fru_matrices.B
    print(f"A:\n", A)
    print(f"Gu's A:\n", gu_A)
    print(f"B:\n", B)
    print(f"Gu's B:\n", gu_B)
    assert jnp.allclose(A, gu_A)
    assert jnp.allclose(B, gu_B)

In [18]:
test_FRU(N=num_of_coef)

A:
 [[ -1.         -0.         -1.4142135 ...   0.         -1.4142135   0.       ]
 [  0.          0.          0.        ...   0.          0.          0.       ]
 [ -1.4142135   0.         -2.        ...   0.         -2.          0.       ]
 ...
 [  0.          0.          0.        ...   0.          0.          0.       ]
 [ -1.4142135   0.         -2.        ...   0.         -2.        -75.398224 ]
 [  0.          0.          0.        ...   0.         75.398224    0.       ]]
Gu's A:
 [[ -1.          0.         -1.4142135 ...   0.         -1.4142135   0.       ]
 [  0.          0.          0.        ...   0.          0.          0.       ]
 [ -1.4142135   0.         -1.9999999 ...   0.         -1.9999999   0.       ]
 ...
 [  0.          0.          0.        ...   0.          0.          0.       ]
 [ -1.4142135   0.         -1.9999999 ...   0.         -1.9999999 -75.398224 ]
 [  0.          0.          0.        ...   0.         75.398224    0.       ]]
B:
 [[1.       ]
 [0.      

### Truncated Fourier (FouT)

In [19]:
def test_FouT(N):
    fout_matrices = TransMatrix(N=N, measure="fout")
    A, B = fout_matrices.A, fout_matrices.B
    gu_fout_matrices = GuTransMatrix(N=N, measure="fout")
    gu_A, gu_B = gu_fout_matrices.A, gu_fout_matrices.B
    print(f"A:\n", A)
    print(f"Gu's A:\n", gu_A)
    print(f"B:\n", B)
    print(f"Gu's B:\n", gu_B)
    assert jnp.allclose(A, gu_A)
    assert jnp.allclose(B, gu_B)

In [20]:
test_FouT(N=num_of_coef)

A:
 [[  -2.         -0.         -2.828427 ...    0.         -2.828427    0.      ]
 [   0.          0.          0.       ...    0.          0.          0.      ]
 [  -2.828427    0.         -4.       ...    0.         -4.          0.      ]
 ...
 [   0.          0.          0.       ...    0.          0.          0.      ]
 [  -2.828427    0.         -4.       ...    0.         -4.       -150.79645 ]
 [   0.          0.          0.       ...    0.        150.79645     0.      ]]
Gu's A:
 [[  -2.           0.          -2.828427  ...    0.          -2.828427     0.       ]
 [   0.           0.           0.        ...    0.           0.           0.       ]
 [  -2.828427     0.          -3.9999998 ...    0.          -3.9999998    0.       ]
 ...
 [   0.           0.           0.        ...    0.           0.           0.       ]
 [  -2.828427     0.          -3.9999998 ...    0.          -3.9999998 -150.79645  ]
 [   0.           0.           0.        ...    0.         150.79645      0. 

### Fourier With Decay (FourD)

In [21]:
def test_FouD(N):
    the_measure = "foud"
    foud_matrices = TransMatrix(N=N, measure="foud")
    A, B = foud_matrices.A, foud_matrices.B
    gu_foud_matrices = GuTransMatrix(N=N, measure="foud")
    gu_A, gu_B = gu_foud_matrices.A, gu_foud_matrices.B
    print(f"A:\n", A)
    print(f"Gu's A:\n", gu_A)
    print(f"B:\n", B)
    print(f"Gu's B:\n", gu_B)
    assert jnp.allclose(A, gu_A)
    assert jnp.allclose(B, gu_B)

In [22]:
test_FouD(N=num_of_coef)

A:
 [[ -0.5         -0.          -0.70710677 ...   0.          -0.70710677   0.        ]
 [  0.           0.           0.         ...   0.           0.           0.        ]
 [ -0.70710677   0.          -1.         ...   0.          -1.           0.        ]
 ...
 [  0.           0.           0.         ...   0.           0.           0.        ]
 [ -0.70710677   0.          -1.         ...   0.          -1.         -75.398224  ]
 [  0.           0.           0.         ...   0.          75.398224     0.        ]]
Gu's A:
 [[ -0.5          0.          -0.70710677 ...   0.          -0.70710677   0.        ]
 [  0.           0.           0.         ...   0.           0.           0.        ]
 [ -0.70710677   0.          -0.99999994 ...   0.          -0.99999994   0.        ]
 ...
 [  0.           0.           0.         ...   0.           0.           0.        ]
 [ -0.70710677   0.          -0.99999994 ...   0.          -0.99999994 -75.398224  ]
 [  0.           0.           0.         

## Gu's HiPPO Linear Time Invariant Operator

In [23]:
class HiPPO_LTI(nn.Module):
    """Linear time invariant x' = Ax + Bu"""

    def __init__(
        self,
        N,
        method="legt",
        dt=1.0,
        T=1.0,
        discretization=0.5,
        lambda_n=1.0,
        alpha=0.0,
        beta=1.0,
        c=0.0,
    ):
        """
        N: the order of the HiPPO projection
        dt: discretization step size - should be roughly inverse to the length of the sequence
        """
        super().__init__()

        self.method = method
        self.N = N
        self.dt = dt
        self.T = T
        self.c = c

        matrices = GuTransMatrix(
            N=N, measure=method, lambda_n=lambda_n, alpha=alpha, beta=beta
        )
        A = np.asarray(matrices.A, dtype=np.float32)
        B = np.asarray(matrices.B, dtype=np.float32)
        # A, B = transition(method, N)
        A = A + (np.eye(N) * c)
        self.A = A
        self.B = B.squeeze(-1)
        self.measure_fn = measure(method)

        C = np.ones((1, N))
        D = np.zeros((1,))
        if type(discretization) in [float, int]:
            dA, dB, _, _, _ = signal.cont2discrete(
                (A, B, C, D), dt=dt, method="gbt", alpha=discretization
            )
        else:
            dA, dB, _, _, _ = signal.cont2discrete((A, B, C, D), dt=dt, method="zoh")

        dB = dB.squeeze(-1)

        self.dA = torch.Tensor(dA.copy())  # (N, N)
        self.dB = torch.Tensor(dB.copy())  # (N, )

        self.vals = np.arange(0.0, T, dt)
        self.eval_matrix = basis(self.method, self.N, self.vals, c=self.c)  # (T/dt, N)
        self.measure = measure(self.method)(self.vals)

    def forward(self, inputs, fast=True):
        """
        inputs : (length, ...)
        output : (length, ..., N) where N is the order of the HiPPO projection
        """

        inputs = inputs.unsqueeze(-1)
        u = inputs * self.dB  # (length, ..., N)

        if fast:
            dA = repeat(self.dA, "m n -> l m n", l=u.size(0))
            return variable_unroll_matrix(dA, u)

        c = torch.zeros(u.shape[1:]).to(inputs)
        cs = []
        for f in inputs:

            # print(f"dA shape:\n{self.dA.shape}")
            # print(f"dA:\n{self.dA}")

            # print(f"c shape:\n{c.shape}")
            # print(f"c:\n{c}")

            # print(f"dB shape:\n{self.dB.shape}")
            # print(f"dB:\n{self.dB}")

            # print(f"f shape:\n{f.shape}")
            # print(f"f:\n{f}")

            part1 = F.linear(c, self.dA)
            part2 = self.dB * f

            c = part1 + part2

            # print(f"part1 shape:\n{part1.shape}")
            # print(f"part1 :\n{part1}")

            # print(f"part2 shape:\n{part2.shape}")
            # print(f"part2:\n{part2}")

            cs.append(c)
        return torch.stack(cs, dim=0)

    def reconstruct(
        self, c, evals=None
    ):  # TODO take in a times array for reconstruction
        """
        c: (..., N,) HiPPO coefficients (same as x(t) in S4 notation)
        output: (..., L,)
        """
        if evals is not None:
            eval_matrix = basis(self.method, self.N, evals)
        else:
            eval_matrix = self.eval_matrix

        m = self.measure[self.measure != 0.0]

        c = c.unsqueeze(-1)
        y = eval_matrix.to(c) @ c
        return y.squeeze(-1).flip(-1)

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

## Gu's Scale invariant HiPPO LegS Operator

In [24]:
class HiPPO_LSI(nn.Module):
    """Vanilla HiPPO-LegS model (scale invariant instead of time invariant)"""

    def __init__(
        self,
        N,
        method="legs",
        max_length=1024,
        discretization=0.5,
        lambda_n=1.0,
        alpha=0.0,
        beta=1.0,
    ):
        """
        max_length: maximum sequence length
        """
        super().__init__()
        self.N = N
        matrices = GuTransMatrix(
            N=N, measure=method, lambda_n=lambda_n, alpha=alpha, beta=beta
        )
        A = np.asarray(matrices.A, dtype=np.float32)
        B = np.asarray(matrices.B, dtype=np.float32)
        # A, B = transition(method, N)
        B = B.squeeze(-1)
        A_stacked = np.empty((max_length, N, N), dtype=A.dtype)
        B_stacked = np.empty((max_length, N), dtype=B.dtype)
        for t in range(1, max_length + 1):
            At = A / t
            Bt = B / t
            if discretization == 0.0:  # forward
                A_stacked[t - 1] = np.eye(N) + At
                B_stacked[t - 1] = Bt
            elif discretization == 1.0:  # backward
                A_stacked[t - 1] = la.solve_triangular(
                    np.eye(N) - At, np.eye(N), lower=True
                )
                B_stacked[t - 1] = la.solve_triangular(np.eye(N) - At, Bt, lower=True)
            elif discretization == 0.5:  # bilinear
                # A_stacked[t - 1] = la.solve_triangular(
                #     np.eye(N) - At / 2, np.eye(N) + At / 2, lower=True
                # )
                # B_stacked[t - 1] = la.solve_triangular(
                #     np.eye(N) - At / 2, Bt, lower=True
                # )
                alpha = 0.5
                A_stacked[t - 1] = np.linalg.lstsq(
                    np.eye(N) - (At * alpha), np.eye(N) + (At * alpha), rcond=None
                )[
                    0
                ]  # TODO: Referencing this: https://stackoverflow.com/questions/64527098/numpy-linalg-linalgerror-singular-matrix-error-when-trying-to-solve
                B_stacked[t - 1] = np.linalg.lstsq(
                    np.eye(N) - (At * alpha), Bt, rcond=None
                )[0]
            else:  # ZOH
                A_stacked[t - 1] = la.expm(A * (math.log(t + 1) - math.log(t)))
                # A_stacked[t - 1] = la.expm(At)
                B_stacked[t - 1] = la.solve_triangular(
                    A, A_stacked[t - 1] @ B - B, lower=True
                )

                # A_stacked[t - 1] = la.expm(At)
                # B_stacked[t - 1] = la.inv(A) @ (la.expm(At) - np.eye(A.shape[0])) @ B

        # self.register_buffer('A_stacked', torch.Tensor(A_stacked)) # (max_length, N, N)
        # self.register_buffer('B_stacked', torch.Tensor(B_stacked)) # (max_length, N)

        self.A_stacked = torch.Tensor(A_stacked.copy())  # (max_length, N, N)
        self.B_stacked = torch.Tensor(B_stacked.copy())  # (max_length, N)

        vals = np.linspace(0.0, 1.0, max_length)
        self.eval_matrix = torch.from_numpy(
            np.asarray(
                ((B[:, None] * ss.eval_legendre(np.arange(N)[:, None], 2 * vals - 1)).T)
            )
        )

    def forward(self, inputs, fast=True):
        """
        inputs : (length, ...)
        output : (length, ..., N) where N is the order of the HiPPO projection
        """

        L = inputs.shape[0]

        inputs = inputs.unsqueeze(-1)
        u = torch.transpose(inputs, 0, -2)
        u = u * self.B_stacked[:L]
        # print(f"Gu - u * self.B_stacked[:L]: {u}")
        u = torch.transpose(u, 0, -2)  # (length, ..., N)

        if fast:
            result = variable_unroll_matrix(self.A_stacked[:L], u)
            return result

        c = torch.zeros(u.shape[1:]).to(inputs)
        cs = []
        for t, f in enumerate(inputs):
            # print(f"\n--------------step {t}----------------")
            # print(f"self.A_stacked[{t}] shape:\n{self.A_stacked[t].shape}")
            # print(f"self.A_stacked[{t}]:\n{self.A_stacked[t]}")

            # print(f"c shape:\n{c.shape}")
            # print(f"c:\n{c}")

            # print(f"self.B_stacked[{t}] shape:\n{self.B_stacked[t].shape}")
            # print(f"self.B_stacked[{t}]:\n{self.B_stacked[t]}")

            # print(f"f shape:\n{f.shape}")
            # print(f"f:\n{f}")

            part1 = F.linear(c, self.A_stacked[t])
            part2 = self.B_stacked[t] * f

            c = part1 + part2

            # print(f"part1 - {t} - shape:\n{part1.shape}")
            # print(f"part1 - {t} -:\n{part1}")

            # print(f"part2 - {t} - shape:\n{part2.shape}")
            # print(f"part2 - {t} -:\n{part2}")

            cs.append(c)
        return torch.stack(cs, dim=0)

    def reconstruct(self, c):
        a = self.eval_matrix.to(c) @ c.unsqueeze(-1)
        return a

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

## Implementation Of General HiPPO Operator

In [25]:
class HiPPO(jnn.Module):
    """
    class that constructs HiPPO model using the defined measure.

    Args:

        max_length (int):
            maximum sequence length to be input

        step_size (float):
            step size used for descretization

        N (int):
            order of the HiPPO projection, aka the number of coefficients to describe the matrix

        lambda_n (float):
            value associated with the tilt of legt
            - 1: tilt on legt
            - \sqrt(2n+1)(-1)^{N}: tilt associated with the legendre memory unit (LMU)

        alpha (float):
            The order of the Laguerre basis.

        beta (float):
            The scale of the Laguerre basis.

        GBT_alpha (float):
            represents which descretization transformation to use based off the alpha value

        measure (str):
            the measure used to define which way to instantiate the HiPPO matrix

        s_t (str):
            choice between LSI and LTI systems
            - "lsi"
            - "lti"

        dtype (jnp.float):
            represents the float precision of the class

        verbose (bool):
            shows the rolled out coefficients over time/scale

    """

    max_length: int
    step_size: float = 1.0  # < 1.0 if you want to use LTI discretization
    N: int = 100
    lambda_n: float = 1.0
    alpha: float = 0.0
    beta: float = 1.0
    GBT_alpha: float = 0.5
    measure: str = "legs"
    s_t: str = "lti"
    dtype: Any = jnp.float32
    verbose: bool = False

    def setup(self):
        matrices = TransMatrix(
            N=self.N,
            measure=self.measure,
            lambda_n=self.lambda_n,
            alpha=self.alpha,
            beta=self.beta,
            dtype=self.dtype,
        )

        self.A = matrices.A
        self.B = matrices.B

        self.C = jnp.ones((self.N, 1))
        self.D = jnp.zeros((1,))

        if self.step_size == 1.0:
            self.GBT_A_list, self.GBT_B_list = self.make_GBT_list(
                matrices.A, matrices.B, dtype=self.dtype
            )

        self.eval_matrix = self.create_eval_matrix(matrices.A, matrices.B)

    def __call__(self, f, init_state=None, kernel=False):
        if not kernel:
            if init_state is None:
                init_state = jnp.zeros((f.shape[0], 1, self.N))

            if self.s_t == "lsi":
                c_k, y_k = self.lsi_recurrence(
                    A=self.GBT_A_list,
                    B=self.GBT_B_list,
                    C=self.C,
                    D=self.D,
                    c_0=init_state,
                    f=f,
                    alpha=self.GBT_alpha,
                    dtype=self.dtype,
                )
                c_k = jnp.stack(c_k, axis=0)
                y_k = jnp.stack(y_k, axis=0)

            elif self.s_t == "lti":
                c_k, y_k = self.lti_recurrence(
                    A=self.A,
                    B=self.B,
                    C=self.C,
                    D=self.D,
                    c_0=init_state,
                    f=f,
                    alpha=self.GBT_alpha,
                    step_size=self.step_size,
                    dtype=self.dtype,
                )
            else:
                raise ValueError(
                    f"Incorrect value associated with invariance options, either pick 'lsi' or 'lti'."
                )

        else:
            Ab, Bb, Cb, Db = self.discretize(
                self.A,
                self.B,
                self.C,
                self.D,
                step=self.step_size,
                alpha=self.GBT_alpha,
            )
            c_k, y_k = self.causal_convolution(
                f, self.K_conv(Ab, Bb, Cb, Db, L=self.max_length)
            )

        return c_k, y_k

    def reconstruct(self, c):
        """
        Uses coeffecients to reconstruct the signal

        Args:
            c (jnp.ndarray): coefficients of the HiPPO projection

        Returns:
            reconstructed signal
        """
        return (self.eval_matrix @ jnp.expand_dims(c, -1)).squeeze(-1)

    def make_GBT_list(self, A, B, dtype=jnp.float32):
        """
        Creates the discretized GBT matrices for the given step size
        """
        GBT_a_list = []
        GBT_b_list = []
        for i in range(1, self.max_length + 1):
            GBT_A, GBT_B = self.discretize(
                A, B, step=i, alpha=self.GBT_alpha, dtype=dtype
            )
            GBT_a_list.append(GBT_A)
            GBT_b_list.append(GBT_B)

        return GBT_a_list, GBT_b_list

    def create_eval_matrix(self, A, B):
        """
        Creates the evaluation matrix used for reconstructing the signal
        """
        eval_matrix = None
        if self.measure == "legs":
            L = self.max_length
            vals = jnp.linspace(0.0, 1.0, L)
            # n = jnp.arange(self.N)[:, None]
            zero_N = self.N - 1
            x = 2 * vals - 1
            eval_matrix = (
                B[:, None]
                * jax.scipy.special.lpmn_values(
                    m=zero_N, n=zero_N, z=x, is_normalized=False
                )
            ).T  # ss.eval_legendre(n, x)).T

        elif self.measure == "legt":
            L = self.max_length
            vals = jnp.arange(0.0, 1.0, L)
            # n = jnp.arange(self.N)[:, None]
            zero_N = self.N - 1
            x = 1 - 2 * vals
            eval_matrix = jax.scipy.special.lpmn_values(
                m=zero_N, n=zero_N, z=x, is_normalized=False
            ).T  # ss.eval_legendre(n, x).T

        elif self.measure == "lmu":
            # raise NotImplementedError("LMU measure not implemented yet")
            pass

        elif self.measure == "lagt":
            # raise NotImplementedError("Translated Laguerre measure not implemented yet")
            pass

        elif self.measure in ["fourier", "fru", "fout", "foud"]:
            # raise NotImplementedError("Fourier measures are not implemented yet")
            pass

        else:
            raise ValueError("invalid measure")

        return eval_matrix

    def discretize(self, A, B, step, alpha=0.5, dtype=jnp.float32):
        """
        function used for discretizing the HiPPO matrix

        Args:
            A (jnp.ndarray):
                shape: (N, N)
                matrix to be discretized

            B (jnp.ndarray):
                shape: (N, 1)
                matrix to be discretized

            C (jnp.ndarray):
                shape: (N, 1)
                matrix to be discretized

            D (jnp.ndarray):
                shape: (1,)
                matrix to be discretized

            step (float):
                step size used for discretization

            alpha (float, optional):
                used for determining which generalized bilinear transformation to use
                - forward Euler corresponds to α = 0,
                - backward Euler corresponds to α = 1,
                - bilinear corresponds to α = 0.5,
                - Zero-order Hold corresponds to α > 1
        """
        if alpha <= 1:
            assert (
                alpha == 0.0 or alpha == 0.5 or alpha == 1.0
            ), "alpha must be 0, 0.5, or 1"
        else:
            assert (
                alpha > 1 or type(alpha) == str
            ), "alpha must be greater than 1 for zero-order hold"

        I = jnp.eye(A.shape[0])

        if alpha <= 1:  # Generalized Bilinear Transformation

            step_size = 1 / step
            part1 = I - (step_size * alpha * A)
            part2 = I + (step_size * (1 - alpha) * A)

            GBT_A = jnp.linalg.lstsq(part1, part2, rcond=None)[0]
            GBT_B = jnp.linalg.lstsq(part1, (step_size * B), rcond=None)[0]

        else:  # Zero-order Hold
            # refer to this for why this works
            # https://en.wikipedia.org/wiki/Discretization#:~:text=A%20clever%20trick%20to%20compute%20Ad%20and%20Bd%20in%20one%20step%20is%20by%20utilizing%20the%20following%20property

            n = A.shape[0]
            b_n = B.shape[1]
            A_B_square = jnp.block(
                [[A, B], [jnp.zeros((b_n, n)), jnp.zeros((b_n, b_n))]]
            )
            if self.s_t == "lsi":
                A_B = jax.scipy.linalg.expm(
                    A_B_square * (math.log(step + self.step_size) - math.log(step))
                )

            else:
                A_B = jax.scipy.linalg.expm(A_B_square * self.step_size)

            GBT_A = A_B[0:n, 0:n]
            GBT_B = A_B[0:-b_n, -b_n:]

        return GBT_A.astype(dtype), GBT_B.astype(dtype)

    def lsi_recurrence(self, A, B, C, D, c_0, f, alpha=0.5, dtype=jnp.float32):
        """
        This is for returning the discretized hidden state often needed for an RNN.
        Args:
            A (jnp.ndarray):
                shape: (N, N)
                the discretized A matrix

            B (jnp.ndarray):
                shape: (N, 1)
                the discretized B matrix

            C (jnp.ndarray):
                shape: (N, 1)
                the discretized C matrix

            c_0 (jnp.ndarray):
                shape: (batch size, input length, N)
                the initial hidden state

            f (jnp.ndarray):
                shape: (sequence length, 1)
                the input sequence


        Returns:
            the next hidden state (aka coefficients representing the function, f(t))
        """

        c_k_list = []
        y_k_list = []

        c_k = c_0.copy()
        for i in range(f.shape[1]):
            c_k, y_k = jax.vmap(self.lsi_step, in_axes=(None, None, None, None, 0, 0))(
                A[i], B[i], C, D, c_k, f[:, i, :]
            )
            c_k_list.append((c_k.copy()).astype(dtype))
            y_k_list.append((y_k.copy()).astype(dtype))

        if self.verbose:
            return c_k_list, y_k_list
        else:
            return c_k_list[-1], y_k_list[-1]

    def lti_recurrence(
        self, A, B, C, D, c_0, f, alpha=0.5, step_size=1.0, dtype=jnp.float32
    ):
        """
        This is for returning the discretized hidden state often needed for an RNN.
        Args:
            A (jnp.ndarray):
                shape: (N, N)
                the discretized A matrix

            B (jnp.ndarray):
                shape: (N, 1)
                the discretized B matrix

            C (jnp.ndarray):
                shape: (N, 1)
                the discretized C matrix

            D (jnp.ndarray):
                shape: (N, 1)
                the discretized C matrix

            f (jnp.ndarray):
                shape: (sequence length, 1)
                the input sequence

            c_0 (jnp.ndarray):
                shape: (batch size, input length, N)
                the initial hidden state

        Returns:
            the next hidden state (aka coefficients representing the function, f(t))
        """
        Ad, Bd = self.discretize(A=A, B=B, step=step_size, alpha=alpha, dtype=dtype)

        def lti_step(c_k_i, f_k):
            """
            Get descretized coefficients of the hidden state by applying HiPPO matrix to input sequence, u_k, and previous hidden state, x_k_1.
            Args:
                c_k_i:
                    shape: (input length, N)
                    previous hidden state

                f_k:
                    shape: (1, )
                    output from function f at, descritized, time step, k.

            Returns:
                c_k: current hidden state
                y_k: current output of hidden state applied to Cb (sorry for being vague, I just dont know yet)
            """

            c_k = (jnp.dot(c_k_i, Ad.T)) + (Bd.T * f_k)
            y_k = (jnp.dot(C, c_k)) + (D * f_k)

            return c_k, (c_k, y_k)

        c_k, (c_s, y_s) = jax.vmap(jax.lax.scan, in_axes=(None, 0, 0))(lti_step, c_0, f)

        if self.verbose:
            return c_s, y_s
        else:
            return c_k, y_s

    def lsi_step(self, Ad, Bd, Cd, Dd, c_k_i, f_k):
        """
        Get descretized coefficients of the hidden state by applying HiPPO matrix to input sequence, u_k, and previous hidden state, x_k_1.
        Args:
            c_k_i:
                shape: (input length, N)
                previous hidden state

            f_k:
                shape: (1, )
                output from function f at, descritized, time step, k.

        Returns:
            c_k: current hidden state
            y_k: current output of hidden state applied to Cb (sorry for being vague, I just dont know yet)
        """

        c_k = (jnp.dot(c_k_i, Ad.T)) + (Bd.T * f_k)
        y_k = (jnp.dot(Cd, c_k)) + (Dd * f_k)

        return c_k, y_k

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

In [26]:
def random_16_input(key_generator, batch_size=16, data_size=784, input_size=28):
    # x = jax.random.randint(key_generator, (batch_size, data_size), 0, 255)
    x = jax.random.uniform(key_generator, (batch_size, data_size))
    return np.asarray(jax.vmap(moving_window, in_axes=(0, None))(x, input_size))

## Test Generalized Bilinear Transform and Zero Order Hold Matrices

In [27]:
def test_LSI_GBT(hippo, gu_hippo, A, B, random_input, alpha=0.5):
    L = random_input.shape[1]
    for i in range(1, L + 1):
        GBT_A, GBT_B = hippo.discretize(A, B, step=i, alpha=alpha, dtype=jnp.float32)
        gu_GBT_A, gu_GBT_B = (
            jnp.asarray(gu_hippo.A_stacked[i - 1], dtype=jnp.float32),
            jnp.expand_dims(
                jnp.asarray(gu_hippo.B_stacked[i - 1], dtype=jnp.float32), axis=1
            ),
        )

        print(f"GBT_A: {jnp.allclose(GBT_A, gu_GBT_A, rtol=1e-04, atol=1e-04)}")
        print(f"GBT_B: {jnp.allclose(GBT_B, gu_GBT_B, rtol=1e-04, atol=1e-04)}\n")

In [28]:
def test_LTI_GBT(hippo, gu_hippo, A, B, random_input, alpha=0.5, print_all=False):
    L = random_input.shape[1]
    GBT_A, GBT_B = hippo.discretize(A, B, step=1.0, alpha=alpha, dtype=jnp.float32)
    gu_GBT_A, gu_GBT_B = (
        jnp.asarray(gu_hippo.dA, dtype=jnp.float32),
        jnp.expand_dims(jnp.asarray(gu_hippo.dB, dtype=jnp.float32), axis=1),
    )
    if print_all:
        print(f"gu_GBT_A shape:{gu_GBT_A.shape}\n")
        print(f"GBT_A shape: {GBT_A.shape}\n")
        print(f"gu_GBT_B shape: {gu_GBT_B.shape}\n")
        print(f"GBT_B shape: {GBT_B.shape}")

        print(f"gu_GBT_A:\n{gu_GBT_A}\n")
        print(f"GBT_A:\n{GBT_A}\n")
        print(f"gu_GBT_B:\n{gu_GBT_B}\n")
        print(f"GBT_B:\n{GBT_B}")

    print(f"GBT_A: {jnp.allclose(GBT_A, gu_GBT_A, rtol=1e-04, atol=1e-04)}")
    print(f"GBT_B: {jnp.allclose(GBT_B, gu_GBT_B, rtol=1e-04, atol=1e-04)}\n")

In [29]:
def test_GBT(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=False
):
    batch_size = 16
    data_size = 256
    input_size = 1

    N = 50
    L = data_size

    x_np = random_16_input(
        key_generator=key3,
        batch_size=batch_size,
        data_size=data_size,
        input_size=input_size,
    )
    print(x_np.shape)

    print(f"Creating Gu's HiPPO-{the_measure} LTI model with {alpha} transform")
    gu_hippo_lti = HiPPO_LTI(
        N=N,
        method=the_measure,
        dt=1.0,
        T=1.0,
        discretization=discretization,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        c=0.0,
    )  # The Gu's

    if the_measure == "legs":
        print(f"Creating Gu's HiPPO-{the_measure} LSI model with {alpha} transform")
        gu_hippo_lsi = HiPPO_LSI(
            N=N,
            method="legs",
            max_length=L,
            discretization=discretization,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
        )  # The Gu's

    matrices = TransMatrix(
        N=N,
        measure=the_measure,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        dtype=jnp.float32,
    )

    A = matrices.A
    B = matrices.B

    print(f"Creating HiPPO-{the_measure} LTI model with {alpha} transform")
    hippo_lti = HiPPO(
        max_length=L,
        step_size=1.0,
        N=N,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        GBT_alpha=alpha,
        measure=the_measure,
        s_t="lti",
        dtype=jnp.float32,
        verbose=True,
    )  # Bryan's

    if the_measure == "legs":
        print(f"Creating HiPPO-{the_measure} LSI model with {alpha} transform")
        hippo_lsi = HiPPO(
            max_length=L,
            step_size=1.0,
            N=N,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
            GBT_alpha=alpha,
            measure=the_measure,
            s_t="lsi",
            dtype=jnp.float32,
            verbose=True,
        )  # Bryan's

    print(f"Testing for correct LTI GBT matrices for HiPPO-{the_measure}")
    test_LTI_GBT(
        hippo=hippo_lti,
        gu_hippo=gu_hippo_lti,
        A=A,
        B=B,
        random_input=x_np,
        alpha=alpha,
        print_all=print_all,
    )
    if the_measure == "legs":
        print(f"Testing for correct LSI GBT matrices for HiPPO-{the_measure}")
        test_LSI_GBT(
            hippo=hippo_lsi,
            gu_hippo=gu_hippo_lsi,
            A=A,
            B=B,
            random_input=x_np,
            alpha=alpha,
        )

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

In [30]:
print_all = False

### Testing Forward Euler Transform for LTI and LSI

#### LegS

In [31]:
test_GBT(
    the_measure="legs", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-legs LTI model with 0.0 transform
Creating Gu's HiPPO-legs LSI model with 0.0 transform
Creating HiPPO-legs LTI model with 0.0 transform
Creating HiPPO-legs LSI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-legs
GBT_A: True
GBT_B: True

Testing for correct LSI GBT matrices for HiPPO-legs
GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GB

#### LegT

In [32]:
test_GBT(
    the_measure="legt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-legt LTI model with 0.0 transform
Creating HiPPO-legt LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-legt
GBT_A: True
GBT_B: True



#### LMU

In [33]:
test_GBT(
    the_measure="lmu", lambda_n=2.0, alpha=0.0, discretization=0.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-lmu LTI model with 0.0 transform
Creating HiPPO-lmu LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-lmu
GBT_A: True
GBT_B: True



#### LagT

In [34]:
test_GBT(
    the_measure="lagt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-lagt LTI model with 0.0 transform
Creating HiPPO-lagt LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-lagt
GBT_A: True
GBT_B: True



#### FRU

In [35]:
test_GBT(
    the_measure="fru", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-fru LTI model with 0.0 transform
Creating HiPPO-fru LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-fru
GBT_A: True
GBT_B: True



#### FouT

In [36]:
test_GBT(
    the_measure="fout", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-fout LTI model with 0.0 transform
Creating HiPPO-fout LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-fout
GBT_A: True
GBT_B: True



#### FouD

In [37]:
test_GBT(
    the_measure="foud", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-foud LTI model with 0.0 transform
Creating HiPPO-foud LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-foud
GBT_A: True
GBT_B: True



### Testing Backward Euler Transform for LTI and LSI on LegS Matrices

#### LegS

In [38]:
test_GBT(
    the_measure="legs", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-legs LTI model with 1.0 transform
Creating Gu's HiPPO-legs LSI model with 1.0 transform
Creating HiPPO-legs LTI model with 1.0 transform
Creating HiPPO-legs LSI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-legs
GBT_A: True
GBT_B: True

Testing for correct LSI GBT matrices for HiPPO-legs
GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GB

#### LegT

In [39]:
test_GBT(
    the_measure="legt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-legt LTI model with 1.0 transform
Creating HiPPO-legt LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-legt
GBT_A: True
GBT_B: True



#### LMU

In [40]:
test_GBT(
    the_measure="lmu", lambda_n=2.0, alpha=1.0, discretization=1.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-lmu LTI model with 1.0 transform
Creating HiPPO-lmu LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-lmu
GBT_A: True
GBT_B: True



#### LagT

In [41]:
test_GBT(
    the_measure="lagt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-lagt LTI model with 1.0 transform
Creating HiPPO-lagt LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-lagt
GBT_A: True
GBT_B: True



#### FRU

In [42]:
test_GBT(
    the_measure="fru", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-fru LTI model with 1.0 transform
Creating HiPPO-fru LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-fru
GBT_A: True
GBT_B: True



#### FouT

In [43]:
test_GBT(
    the_measure="fout", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-fout LTI model with 1.0 transform
Creating HiPPO-fout LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-fout
GBT_A: True
GBT_B: True



#### FouD

In [44]:
test_GBT(
    the_measure="foud", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-foud LTI model with 1.0 transform
Creating HiPPO-foud LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-foud
GBT_A: True
GBT_B: True



### Testing Bidirectional Transform for LTI and LSI on LegS Matrices

#### LegS

In [45]:
test_GBT(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-legs LTI model with 0.5 transform
Creating Gu's HiPPO-legs LSI model with 0.5 transform
Creating HiPPO-legs LTI model with 0.5 transform
Creating HiPPO-legs LSI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-legs
GBT_A: True
GBT_B: True

Testing for correct LSI GBT matrices for HiPPO-legs
GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GB

#### LegT

In [46]:
test_GBT(
    the_measure="legt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-legt LTI model with 0.5 transform
Creating HiPPO-legt LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-legt
GBT_A: True
GBT_B: True



#### LMU

In [47]:
test_GBT(
    the_measure="lmu", lambda_n=2.0, alpha=0.5, discretization=0.5, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-lmu LTI model with 0.5 transform
Creating HiPPO-lmu LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-lmu
GBT_A: True
GBT_B: True



#### LagT

In [48]:
test_GBT(
    the_measure="lagt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-lagt LTI model with 0.5 transform
Creating HiPPO-lagt LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-lagt
GBT_A: True
GBT_B: True



#### FRU

In [49]:
test_GBT(
    the_measure="fru", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-fru LTI model with 0.5 transform
Creating HiPPO-fru LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-fru
GBT_A: True
GBT_B: True



#### FouT

In [50]:
test_GBT(
    the_measure="fout", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-fout LTI model with 0.5 transform
Creating HiPPO-fout LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-fout
GBT_A: True
GBT_B: True



#### FouD

In [51]:
test_GBT(
    the_measure="foud", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

(16, 256, 1)
Creating Gu's HiPPO-foud LTI model with 0.5 transform
Creating HiPPO-foud LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-foud
GBT_A: True
GBT_B: True



### Testing ZOH Transform for LTI and LSI on LegS Matrices

#### LegS

In [52]:
test_GBT(
    the_measure="legs",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

(16, 256, 1)
Creating Gu's HiPPO-legs LTI model with 2.0 transform
Creating Gu's HiPPO-legs LSI model with 2.0 transform
Creating HiPPO-legs LTI model with 2.0 transform
Creating HiPPO-legs LSI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-legs
GBT_A: True
GBT_B: True

Testing for correct LSI GBT matrices for HiPPO-legs
GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GBT_A: True
GBT_B: True

GB

#### LegT

In [53]:
test_GBT(
    the_measure="legt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

(16, 256, 1)
Creating Gu's HiPPO-legt LTI model with 2.0 transform
Creating HiPPO-legt LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-legt
GBT_A: True
GBT_B: True



#### LMU

In [54]:
test_GBT(
    the_measure="lmu",
    lambda_n=2.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

(16, 256, 1)
Creating Gu's HiPPO-lmu LTI model with 2.0 transform
Creating HiPPO-lmu LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-lmu
GBT_A: True
GBT_B: True



#### LagT

In [55]:
test_GBT(
    the_measure="lagt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

(16, 256, 1)
Creating Gu's HiPPO-lagt LTI model with 2.0 transform
Creating HiPPO-lagt LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-lagt
GBT_A: True
GBT_B: True



#### FRU

In [56]:
test_GBT(
    the_measure="fru",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

(16, 256, 1)
Creating Gu's HiPPO-fru LTI model with 2.0 transform
Creating HiPPO-fru LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-fru
GBT_A: True
GBT_B: True



#### FouT

In [57]:
test_GBT(
    the_measure="fout",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

(16, 256, 1)
Creating Gu's HiPPO-fout LTI model with 2.0 transform
Creating HiPPO-fout LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-fout
GBT_A: True
GBT_B: True



#### FouD

In [58]:
test_GBT(
    the_measure="foud",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

(16, 256, 1)
Creating Gu's HiPPO-foud LTI model with 2.0 transform
Creating HiPPO-foud LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-foud
GBT_A: True
GBT_B: True



## Test HiPPO Operators

In [59]:
def test_hippo_operator(
    hippo_legs, gu_hippo_legs, random_input, key, s_or_t="lti", print_all=False
):
    x_tensor = torch.tensor(random_input, dtype=torch.float32)
    x_jnp = jnp.asarray(x_tensor, dtype=jnp.float32)  # convert torch array to jax array

    # My Implementation
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"----------------------------My {s_or_t} Implementation Outputs----------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    params = hippo_legs.init(key, f=x_jnp)
    c_k, y_k_list = hippo_legs.apply(params, f=x_jnp)
    if s_or_t == "lsi":
        c_k = jnp.moveaxis(c_k, 0, 1)

    # Gu's HiPPO LegS
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"---------------------------Gu's {s_or_t} Implementation Outputs---------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    x_tensor = torch.moveaxis(x_tensor, 0, 1)
    GU_c_k = gu_hippo_legs(x_tensor, fast=False)
    gu_c = jnp.asarray(GU_c_k, dtype=jnp.float32)  # convert torch array to jax array
    gu_c = jnp.moveaxis(gu_c, 0, 1)

    if print_all:
        print(
            f"------------------------------------------------------------------------------"
        )
        print(
            f"---------------------------Testing {s_or_t} Outputs---------------------------"
        )
        print(
            f"------------------------------------------------------------------------------"
        )
        jax.debug.print(f"inputted jnp-data shape: {x_jnp.shape}")
        jax.debug.print(f"inputted tensor-data shape: {x_tensor.shape}")
        print(f"c_k shape: {c_k.shape}")
        print(f"gu_c shape: {gu_c.shape}")

    flag = True
    for i in range(c_k.shape[0]):
        for j in range(c_k.shape[1]):
            if print_all:
                print(f"c_k[{i},{j},:,:]:\n{c_k[i,j,:,:]}")
                print(f"gu_c[{i},{j},:,:]:\n{gu_c[i,j,:,:]}")

            check = jnp.allclose(
                c_k[i, j, :, :], gu_c[i, j, :, :], rtol=1e-04, atol=1e-04
            )
            if check == False:
                flag = False
    if not print_all:
        print(f"The Test Passed: {flag}")

In [60]:
def test_operators(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=False
):
    # N = 256
    # L = 128

    batch_size = 16
    data_size = 512
    input_size = 1

    N = 50
    L = data_size

    x_jnp = random_16_input(
        key_generator=key3,
        batch_size=batch_size,
        data_size=data_size,
        input_size=input_size,
    )
    x_np = np.asarray(x_jnp)

    x = torch.tensor(x_np, dtype=torch.float32)

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate Gu's HiPPOs ---------------------------
    # ----------------------------------------------------------------------------------

    print(f"Creating Gu's HiPPO-{the_measure} LTI model with {alpha} transform")
    gu_hippo_lti = HiPPO_LTI(
        N=N,
        method=the_measure,
        dt=1.0,
        T=1.0,
        discretization=discretization,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        c=0.0,
    )  # The Gu's

    if the_measure == "legs":
        print(f"Creating Gu's HiPPO-{the_measure} LSI model with {alpha} transform")
        gu_hippo_lsi = HiPPO_LSI(
            N=N,
            method=the_measure,
            max_length=L,
            discretization=discretization,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
        )  # The Gu's

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate My HiPPOs -----------------------------
    # ----------------------------------------------------------------------------------
    print(f"\nTesting BRYANS HiPPO-{the_measure} model")

    matrices = TransMatrix(
        N=N,
        measure=the_measure,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        dtype=jnp.float32,
    )

    A = matrices.A
    B = matrices.B

    print(f"Creating HiPPO-{the_measure} LTI model with {alpha} transform")
    hippo_lti = HiPPO(
        max_length=L,
        step_size=1.0,
        N=N,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        GBT_alpha=alpha,
        measure=the_measure,
        s_t="lti",
        dtype=jnp.float32,
        verbose=True,
    )  # Bryan's

    if the_measure == "legs":
        print(f"Creating HiPPO-{the_measure} LSI model with {alpha} transform")
        hippo_lsi = HiPPO(
            max_length=L,
            step_size=1.0,
            N=N,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
            GBT_alpha=alpha,
            measure=the_measure,
            s_t="lsi",
            dtype=jnp.float32,
            verbose=True,
        )  # Bryan's

    # ----------------------------------------------------------------------------------
    # ------------------------------ Test HiPPO Operators ------------------------------
    # ----------------------------------------------------------------------------------

    print(f"Bryan's Coeffiecients for {alpha} LTI HiPPO-{the_measure}")

    test_hippo_operator(
        hippo_legs=hippo_lti,
        gu_hippo_legs=gu_hippo_lti,
        random_input=x_np,
        key=key2,
        s_or_t="lti",
        print_all=print_all,
    )

    if the_measure == "legs":
        print(f"\n\nBryan's Coeffiecients for {alpha} LSI HiPPO-{the_measure}")

        test_hippo_operator(
            hippo_legs=hippo_lsi,
            gu_hippo_legs=gu_hippo_lsi,
            random_input=x_np,
            key=key2,
            s_or_t="lsi",
            print_all=print_all,
        )

    print(f"end of test for HiPPO-{the_measure} model")

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

In [61]:
print_all = False

### Testing (LTI and LSI) Operators With Forward Euler Transform

#### LegS

In [62]:
test_operators(
    the_measure="legs", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.0 transform
Creating Gu's HiPPO-legs LSI model with 0.0 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 0.0 transform
Creating HiPPO-legs LSI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-legs
The Test Passed: False


Bryan's Coeffiecients for 0.0 LSI HiPPO-legs
The Test Passed: False
end of test for HiPPO-legs model


#### LegT

In [63]:
test_operators(
    the_measure="legt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.0 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-legt
The Test Passed: False
end of test for HiPPO-legt model


#### LMU

In [64]:
test_operators(
    the_measure="lmu", lambda_n=2.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.0 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-lmu
The Test Passed: False
end of test for HiPPO-lmu model


#### LagT

In [65]:
test_operators(
    the_measure="lagt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.0 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-lagt
The Test Passed: False
end of test for HiPPO-lagt model


#### FRU

In [66]:
test_operators(
    the_measure="fru", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 0.0 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-fru
The Test Passed: False
end of test for HiPPO-fru model


#### FouT

In [67]:
test_operators(
    the_measure="fout", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.0 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-fout
The Test Passed: False
end of test for HiPPO-fout model


#### FouD

In [68]:
test_operators(
    the_measure="foud", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 0.0 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-foud
The Test Passed: False
end of test for HiPPO-foud model


### Testing (LTI and LSI) Operators With Backward Euler Transform

#### LegS

In [69]:
test_operators(
    the_measure="legs", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 1.0 transform
Creating Gu's HiPPO-legs LSI model with 1.0 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 1.0 transform
Creating HiPPO-legs LSI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-legs
The Test Passed: True


Bryan's Coeffiecients for 1.0 LSI HiPPO-legs
The Test Passed: True
end of test for HiPPO-legs model


#### LegT

In [70]:
test_operators(
    the_measure="legt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 1.0 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-legt
The Test Passed: True
end of test for HiPPO-legt model


#### LMU

In [71]:
test_operators(
    the_measure="lmu", lambda_n=2.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 1.0 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-lmu
The Test Passed: True
end of test for HiPPO-lmu model


#### LagT

In [72]:
test_operators(
    the_measure="lagt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 1.0 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-lagt
The Test Passed: True
end of test for HiPPO-lagt model


#### FRU

In [73]:
test_operators(
    the_measure="fru", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 1.0 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-fru
The Test Passed: False
end of test for HiPPO-fru model


#### FouT

In [74]:
test_operators(
    the_measure="fout", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 1.0 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-fout
The Test Passed: True
end of test for HiPPO-fout model


#### FouD

In [75]:
test_operators(
    the_measure="foud", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 1.0 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-foud
The Test Passed: False
end of test for HiPPO-foud model


### Testing (LTI and LSI) Operators With Bidirectional Transform

#### LegS

In [76]:
test_operators(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.5 transform
Creating Gu's HiPPO-legs LSI model with 0.5 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 0.5 transform
Creating HiPPO-legs LSI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-legs
The Test Passed: True


Bryan's Coeffiecients for 0.5 LSI HiPPO-legs
The Test Passed: True
end of test for HiPPO-legs model


#### LegT

In [77]:
test_operators(
    the_measure="legt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.5 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-legt
The Test Passed: True
end of test for HiPPO-legt model


#### LMU

In [78]:
test_operators(
    the_measure="lmu", lambda_n=2.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.5 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-lmu
The Test Passed: False
end of test for HiPPO-lmu model


#### LagT

In [79]:
test_operators(
    the_measure="lagt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.5 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-lagt
The Test Passed: True
end of test for HiPPO-lagt model


#### FRU

In [80]:
test_operators(
    the_measure="fru", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 0.5 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-fru
The Test Passed: True
end of test for HiPPO-fru model


#### FouT

In [81]:
test_operators(
    the_measure="fout", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.5 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-fout
The Test Passed: False
end of test for HiPPO-fout model


#### FouD

In [82]:
test_operators(
    the_measure="foud", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 0.5 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-foud
The Test Passed: True
end of test for HiPPO-foud model


### Testing (LTI and LSI) Operators With ZOH Transform

#### LegS

In [83]:
test_operators(
    the_measure="legs",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legs LTI model with 2.0 transform
Creating Gu's HiPPO-legs LSI model with 2.0 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 2.0 transform
Creating HiPPO-legs LSI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-legs
The Test Passed: True


Bryan's Coeffiecients for 2.0 LSI HiPPO-legs
The Test Passed: True
end of test for HiPPO-legs model


#### LegT

In [84]:
test_operators(
    the_measure="legt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legt LTI model with 2.0 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-legt
The Test Passed: True
end of test for HiPPO-legt model


#### LMU

In [85]:
test_operators(
    the_measure="lmu",
    lambda_n=2.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lmu LTI model with 2.0 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-lmu
The Test Passed: True
end of test for HiPPO-lmu model


#### LagT

In [86]:
test_operators(
    the_measure="lagt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lagt LTI model with 2.0 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-lagt
The Test Passed: True
end of test for HiPPO-lagt model


#### FRU

In [87]:
test_operators(
    the_measure="fru",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-fru LTI model with 2.0 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-fru
The Test Passed: True
end of test for HiPPO-fru model


#### FouT

In [88]:
test_operators(
    the_measure="fout",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-fout LTI model with 2.0 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-fout
The Test Passed: True
end of test for HiPPO-fout model


#### FouD

In [89]:
test_operators(
    the_measure="foud",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-foud LTI model with 2.0 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-foud
The Test Passed: True
end of test for HiPPO-foud model
